In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split,StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import RobustScaler
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
data=pd.read_csv("9000_pages.csv")

In [3]:
data.drop(data.columns[[0, -1, -2]], axis =  1, inplace = True)

In [4]:
data

,Chuyen muc,Nhu cau,Tinh/Thanh pho,Quan/huyen,Xa/Phuong,Duong/khu vuc,Gia,Ngay dang,Dien tich,Huong,So Tang,So phong,Nha ve sinh,Giay to phap
0,Nhà,Cần bán,Hồ Chí Minh,Quận Phú Nhuận,Phường 14,"Đường Huỳnh Văn Bánh, Phường 14, Quận Phú Nhuậ...","6,9 tỷ",11/05/2021,33m²,NaN,NaN,4.0,3.0,Đã có sổ
1,Đất,Cần bán,Hồ Chí Minh,Quận Bình Thạnh,Phường 17,"Đường Nguyễn Cửu Vân, Phường 17, Quận Bình Thạ...","3,6 tỷ",11/05/2021,95m²,NaN,NaN,NaN,NaN,Đã có sổ
2,"Căn hộ, Chung cư",Cho thuê,Hồ Chí Minh,Huyện Nhà Bè,Xã Phước Kiển,"Đường Nguyễn Hữu Thọ, Xã Phước Kiển, Huyện Nhà...",17 triệu/tháng,11/05/2021,106m²,NaN,NaN,3.0,2.0,Đã có sổ
3,Nhà,Cần bán,Hồ Chí Minh,Huyện Hóc Môn,Xã Xuân Thới Sơn,"Đường Kênh An Hạ, Xã Xuân Thới Sơn, Huyện Hóc ...",595 triệu,11/05/2021,104m²,NaN,NaN,8.0,NaN,Đã có sổ
4,Nhà,Cho thuê,Hồ Chí Minh,Quận Bình Tân,Phường Bình Trị Đông A,"1074, Đường Tỉnh Lộ 10, Phường Bình Trị Đông A...",6 triệu/tháng,11/05/2021,30m²,NaN,NaN,2.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399345,"Căn hộ, Chung cư",Cho thuê,Hồ Chí Minh,Quận 7,Phường Tân Quy,"Đường 36, Phường Tân Quy, Quận 7, Tp Hồ Chí Minh",5.000.000 đ/tháng,15/11/2019,25m²,NaN,NaN,1.0,1.0,NaN
399346,Nhà,Cần bán,Hà Nội,Quận Hoàng Mai,Phường Hoàng Văn Thụ,Đường Hoàng Mai,3.300.000.000 đ,15/11/2019,49m²,Đông,5.0,6.0,4.0,Đã có sổ đỏ/sổ hồng
399347,Nhà,Cần bán,Hồ Chí Minh,Quận Bình Tân,Phường Bình Trị Đông,"Đường Lê Văn Quới, Phường Bình Trị Đông, Quận ...",5.600.000.000 đ,15/11/2019,60m²,NaN,NaN,4.0,NaN,NaN
399348,Nhà,Cần bán,Hồ Chí Minh,Huyện Bình Chánh,Xã Hưng Long,"Đường Hưng Long, Xã Hưng Long, Huyện Bình Chán...",650.000.000 đ,15/11/2019,70m²,NaN,2.0,2.0,2.0,Đã có sổ đỏ/sổ hồng


In [5]:
HN = data.loc[data['Tinh/Thanh pho'] == "Hà Nội"]
HN = HN.loc[HN['Nhu cau'] == "Cần bán"]
HN = HN.loc[HN['Chuyen muc'].isin(["Nhà", "Căn hộ, Chung cư"])]

HN['Duong/khu vuc'] = HN['Duong/khu vuc'].fillna("")
HN['Dien tich'] = HN['Dien tich'].fillna("0")

# HN['Duong/khu vuc'] = HN['Duong/khu vuc'].str.split(", Phường", n=1, expand = True)[0]
# HN['Duong/khu vuc'] = HN['Duong/khu vuc'].str.split(", Xã", n=1, expand = True)[0]
# HN['Duong/khu vuc'] = HN['Duong/khu vuc'].str.split(", Thị trấn", n=1, expand = True)[0]
for i, row in HN.iterrows():
    if ", Phường" in row['Duong/khu vuc']:
        row['Duong/khu vuc'] = row['Duong/khu vuc'].split(", Phường")[0]
    if ", Xã" in row['Duong/khu vuc']:
        row['Duong/khu vuc'] = row['Duong/khu vuc'].split(", Xã")[0]
    if ", Thị trấn" in row['Duong/khu vuc']:
        row['Duong/khu vuc'] = row['Duong/khu vuc'].split(", Thị trấn")[0]
    if ", " in row['Duong/khu vuc']:
        row['Duong/khu vuc'] = row['Duong/khu vuc'].split(", ")[1]
    if "Đường " in row['Duong/khu vuc']:
        row['Duong/khu vuc'] = row['Duong/khu vuc'].split("Đường ")[1]
    row['Duong/khu vuc'] = "Đường " + row['Duong/khu vuc']
    HN.at[i,'Duong/khu vuc'] = row['Duong/khu vuc'].title()
    
    try:
        if "triệu" in row['Gia']:
            row['Gia'] = float(row['Gia'].split(" triệu")[0].replace(",", ".")) * 0.001
        elif "tỷ" in row['Gia']:
            row['Gia'] = float(row['Gia'].split(" tỷ")[0].replace(",", "."))
        elif "đ" in row['Gia']:
            row['Gia'] = float(row['Gia'].split(" đ")[0].replace(".", "")) *  0.000000001
        row['Gia'] = round(row['Gia'], 4)
        HN.at[i,'Gia'] = row['Gia']
    except:
        HN.at[i,'Gia'] = 0
    
    if "m²" in row['Dien tich']:
        row['Dien tich'] = row['Dien tich'].split("m²")[0]
    HN.at[i,'Dien tich'] = row['Dien tich']
    HN.at[i,'Thang'] = row['Ngay dang'].split("/")[1]
    HN.at[i,'Nam'] = row['Ngay dang'].split("/")[-1]
HN

,Chuyen muc,Nhu cau,Tinh/Thanh pho,Quan/huyen,Xa/Phuong,Duong/khu vuc,Gia,Ngay dang,Dien tich,Huong,So Tang,So phong,Nha ve sinh,Giay to phap,Thang,Nam
142,Nhà,Cần bán,Hà Nội,Quận Thanh Xuân,Phường Khương Đình,Đường Vũ Tông Phan,0.0068,11/05/2021,35,NaN,5.0,4.0,5.0,Đã có sổ,05,2021
147,Nhà,Cần bán,Hà Nội,Huyện Chương Mỹ,Xã Trung Hòa,Đường Làng,0.5,11/05/2021,220,NaN,NaN,2.0,1.0,Đã có sổ,05,2021
178,Nhà,Cần bán,Hà Nội,Quận Thanh Xuân,Phường Khương Đình,Đường Bùi Xương Trạch,2.78,11/05/2021,32,NaN,4.0,4.0,4.0,Đã có sổ,05,2021
191,Nhà,Cần bán,Hà Nội,Quận Nam Từ Liêm,Phường Mễ Trì,Đường Đỗ Đức Dục,34,11/05/2021,300,NaN,NaN,1.0,NaN,NaN,05,2021
229,Nhà,Cần bán,Hà Nội,Quận Thanh Xuân,Phường Kim Giang,Đường Vũ Tông Phan,6,11/05/2021,45,NaN,5.0,5.0,5.0,Đã có sổ,05,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399244,Nhà,Cần bán,Hà Nội,Quận Hà Đông,Phường Nguyễn Trãi,Đường Tô Hiệu,2.3,15/11/2019,35,NaN,NaN,4.0,NaN,NaN,11,2019
399245,Nhà,Cần bán,Hà Nội,Quận Hà Đông,Phường Nguyễn Trãi,Đường Tô Hiệu,2.3,15/11/2019,35,NaN,NaN,4.0,NaN,NaN,11,2019
399256,"Căn hộ, Chung cư",Cần bán,Hà Nội,Quận Nam Từ Liêm,Phường Mỹ Đình 1,Đường Lê Đức Thọ,2.7,15/11/2019,96,NaN,NaN,3.0,NaN,NaN,11,2019
399269,Nhà,Cần bán,Hà Nội,Quận Bắc Từ Liêm,Phường Phú Diễn,Đường Hồ Tùng Mậu,6.2,15/11/2019,60,Đông Nam,5.0,6.0,4.0,Đã có sổ đỏ/sổ hồng,11,2019


In [6]:
HN['Gia'] = HN['Gia'].astype(np.float64)
HN['Dien tich'] = HN['Dien tich'].astype(np.float64)

In [7]:
HN.dtypes

Chuyen muc         object
Nhu cau            object
Tinh/Thanh pho     object
Quan/huyen         object
Xa/Phuong          object
Duong/khu vuc      object
Gia               float64
Ngay dang          object
Dien tich         float64
Huong              object
So Tang           float64
So phong          float64
Nha ve sinh       float64
Giay to phap       object
Thang              object
Nam                object
dtype: object

In [8]:
HN.drop(HN.columns[[1, 2, 7]], axis = 1, inplace = True)
HN

,Chuyen muc,Quan/huyen,Xa/Phuong,Duong/khu vuc,Gia,Dien tich,Huong,So Tang,So phong,Nha ve sinh,Giay to phap,Thang,Nam
142,Nhà,Quận Thanh Xuân,Phường Khương Đình,Đường Vũ Tông Phan,0.0068,35.0,NaN,5.0,4.0,5.0,Đã có sổ,05,2021
147,Nhà,Huyện Chương Mỹ,Xã Trung Hòa,Đường Làng,0.5000,220.0,NaN,NaN,2.0,1.0,Đã có sổ,05,2021
178,Nhà,Quận Thanh Xuân,Phường Khương Đình,Đường Bùi Xương Trạch,2.7800,32.0,NaN,4.0,4.0,4.0,Đã có sổ,05,2021
191,Nhà,Quận Nam Từ Liêm,Phường Mễ Trì,Đường Đỗ Đức Dục,34.0000,300.0,NaN,NaN,1.0,NaN,NaN,05,2021
229,Nhà,Quận Thanh Xuân,Phường Kim Giang,Đường Vũ Tông Phan,6.0000,45.0,NaN,5.0,5.0,5.0,Đã có sổ,05,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399244,Nhà,Quận Hà Đông,Phường Nguyễn Trãi,Đường Tô Hiệu,2.3000,35.0,NaN,NaN,4.0,NaN,NaN,11,2019
399245,Nhà,Quận Hà Đông,Phường Nguyễn Trãi,Đường Tô Hiệu,2.3000,35.0,NaN,NaN,4.0,NaN,NaN,11,2019
399256,"Căn hộ, Chung cư",Quận Nam Từ Liêm,Phường Mỹ Đình 1,Đường Lê Đức Thọ,2.7000,96.0,NaN,NaN,3.0,NaN,NaN,11,2019
399269,Nhà,Quận Bắc Từ Liêm,Phường Phú Diễn,Đường Hồ Tùng Mậu,6.2000,60.0,Đông Nam,5.0,6.0,4.0,Đã có sổ đỏ/sổ hồng,11,2019


In [46]:
HN.to_csv('HN.csv', index = False)

In [9]:
for col in HN:
#     print(HN[col].unique())
    print(len(HN[col].unique()))

2
28
298
3080
1858
398
9
20
11
9
7
12
3


In [47]:
HCM = data.loc[data['Tinh/Thanh pho'] == "Hồ Chí Minh"]
HCM = HCM.loc[HCM['Nhu cau'] == "Cần bán"]
HCM = HCM.loc[HCM['Chuyen muc'].isin(["Nhà", "Căn hộ, Chung cư"])]

HCM['Duong/khu vuc'] = HCM['Duong/khu vuc'].fillna("")
HCM['Dien tich'] = HCM['Dien tich'].fillna("0")

# HN['Duong/khu vuc'] = HN['Duong/khu vuc'].str.split(", Phường", n=1, expand = True)[0]
# HN['Duong/khu vuc'] = HN['Duong/khu vuc'].str.split(", Xã", n=1, expand = True)[0]
# HN['Duong/khu vuc'] = HN['Duong/khu vuc'].str.split(", Thị trấn", n=1, expand = True)[0]
for i, row in HCM.iterrows():
    if ", Phường" in row['Duong/khu vuc']:
        row['Duong/khu vuc'] = row['Duong/khu vuc'].split(", Phường")[0]
    if ", Xã" in row['Duong/khu vuc']:
        row['Duong/khu vuc'] = row['Duong/khu vuc'].split(", Xã")[0]
    if ", Thị trấn" in row['Duong/khu vuc']:
        row['Duong/khu vuc'] = row['Duong/khu vuc'].split(", Thị trấn")[0]
    if ", " in row['Duong/khu vuc']:
        row['Duong/khu vuc'] = row['Duong/khu vuc'].split(", ")[1]
    if "Đường " in row['Duong/khu vuc']:
        row['Duong/khu vuc'] = row['Duong/khu vuc'].split("Đường ")[1]
    row['Duong/khu vuc'] = "Đường " + row['Duong/khu vuc']
    HCM.at[i,'Duong/khu vuc'] = row['Duong/khu vuc'].title()
    
    try:
        if "triệu" in row['Gia']:
            row['Gia'] = float(row['Gia'].split(" triệu")[0].replace(",", ".")) * 0.001
        elif "tỷ" in row['Gia']:
            row['Gia'] = float(row['Gia'].split(" tỷ")[0].replace(",", "."))
        elif "đ" in row['Gia']:
            row['Gia'] = float(row['Gia'].split(" đ")[0].replace(".", "")) *  0.000000001
        row['Gia'] = round(row['Gia'], 4)

        HCM.at[i,'Gia'] = row['Gia']
    except:
        HCM.at[i,'Gia'] = 0
    
    if "m²" in row['Dien tich']:
        row['Dien tich'] = row['Dien tich'].split("m²")[0]
    if "." in row['Dien tich']:
        row['Dien tich'] = row['Dien tich'].replace(".", "")
    HCM.at[i,'Dien tich'] = row['Dien tich']
    HCM.at[i,'Thang'] = row['Ngay dang'].split("/")[1]
    HCM.at[i,'Nam'] = row['Ngay dang'].split("/")[-1]

HCM['Gia'] = HCM['Gia'].astype(np.float64)
HCM['Dien tich'] = HCM['Dien tich'].astype(np.float64)

HCM.drop(HCM.columns[[1, 2, 7]], axis = 1, inplace = True)
HCM

,Chuyen muc,Quan/huyen,Xa/Phuong,Duong/khu vuc,Gia,Dien tich,Huong,So Tang,So phong,Nha ve sinh,Giay to phap,Thang,Nam
0,Nhà,Quận Phú Nhuận,Phường 14,Đường Huỳnh Văn Bánh,6.900,33.0,NaN,NaN,4.0,3.0,Đã có sổ,05,2021
3,Nhà,Huyện Hóc Môn,Xã Xuân Thới Sơn,Đường Kênh An Hạ,0.595,104.0,NaN,NaN,8.0,NaN,Đã có sổ,05,2021
5,Nhà,Quận Gò Vấp,Phường 10,Đường Quang Trung,4.750,56.0,Đông Nam,3.0,3.0,3.0,Đã có sổ,05,2021
7,Nhà,Quận 5,Phường 1,Đường Trần Bình Trọng,6.000,30.0,NaN,NaN,2.0,1.0,Đã có sổ,05,2021
10,Nhà,Huyện Củ Chi,Xã Phú Hòa Đông,Đường Cây Bài,1.900,125.0,NaN,NaN,2.0,1.0,Đã có sổ,05,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399284,Nhà,Quận 1,Phường Tân Định,Đường Trần Quang Khải,11.500,23.0,NaN,NaN,3.0,3.0,Đã có sổ đỏ/sổ hồng,11,2019
399285,Nhà,Quận Bình Tân,Phường Bình Hưng Hòa,Đường Số 11,6.300,65.0,Tây Nam,NaN,6.0,6.0,Đã có sổ đỏ/sổ hồng,11,2019
399347,Nhà,Quận Bình Tân,Phường Bình Trị Đông,Đường Lê Văn Quới,5.600,60.0,NaN,NaN,4.0,NaN,NaN,11,2019
399348,Nhà,Huyện Bình Chánh,Xã Hưng Long,Đường Hưng Long,0.650,70.0,NaN,2.0,2.0,2.0,Đã có sổ đỏ/sổ hồng,11,2019


In [49]:
for col in HCM:
    #print(HN[col].unique())
    print(len(HCM[col].unique()))

2
26
176
7550
2862
775
9
25
11
8
7
12
3


In [50]:
HCM.to_csv('HCM.csv', index = False)